In [1]:
import os
import sys
import argparse
import numpy as np
import pandas as pd
import tensorflow as tf
from google.cloud import bigquery

In [2]:
import tensorflow_recommenders as tfrs

In [3]:
sys.path.append(os.path.abspath(".."))

In [4]:
from anime_rec_pkg.anime_rec.data.bq_queries.anime_anime_ml_data_queries import anime_anime_retrieval_query

In [5]:
def load_big_query_data(query):
    client = bigquery.Client(project="anime-rec-dev")
    dataset_ref = client.dataset("processed_area")
    job_config = bigquery.QueryJobConfig()
    query_job = client.query(query, job_config=job_config)
    data = query_job.to_dataframe()
    return data

# Retrieval Data

In [6]:
val_data = load_big_query_data(anime_anime_retrieval_query('TEST'))

In [7]:
val_data.head()

,anime_id,retrieved_anime_id
0,21339,40858
1,40165,40858
2,38663,40858
3,40295,40858
4,40269,40858


In [8]:
val_data.shape

(32540, 2)

In [9]:
anime_data = load_big_query_data("SELECT anime_id, genres FROM `anime-rec-dev.processed_area.anime`")

In [10]:
anime_data.head()

,anime_id,genres
0,42447,"[Comedy, Fantasy]"
1,37904,"[Gourmet, Slice of Life, Seinen]"
2,10797,"[Comedy, School]"
3,34207,"[Comedy, Kids]"
4,43772,"[Adventure, Drama, Fantasy, Historical, Martia..."


In [11]:
anime_data.shape

(9792, 2)

In [13]:
val_ds = (
    tf.data.Dataset.from_tensor_slices(
        (
            {
                'animeA' : tf.cast(val_data['anime_id'], tf.string),
                'animeB' : tf.cast(val_data['retrieved_anime_id'], tf.string),
            }
        )
    )
)
val_ds = val_ds.batch(2048).cache()

# Genres

In [14]:
anime_genre_vocab = [
    'Action',
    'Adventure',
    'Avant Garde',
    'Comedy',
    'Demons',
    'Drama',
    'Ecchi',
    'Fantasy',
    'Game',
    'Gourmet',
    'Harem',
    'Historical',
    'Horror',
    'Kids',
    'Martial Arts',
    'Mecha',
    'Military',
    'Mystery',
    'Parody',
    'Police',
    'Psychological',
    'Romance',
    'Samurai',
    'School',
    'Sci-Fi',
    'Seinen',
    'Shoujo',
    'Shounen',
    'Slice of Life',
    'Space',
    'Sports',
    'Super Power',
    'Supernatural',
    'Suspense',
    'Vampire'
]

# Models

In [15]:
class AnimeMultiHotModel(tf.keras.Model):
    def __init__(self,
                unique_anime_ids,
                multi_hot_feature,
                vocabulary):
        super().__init__()

        self.anime_id_lookup_layer = tf.keras.layers.StringLookup(
            vocabulary = unique_anime_ids, 
            num_oov_indices = 0,
            name = 'anime_multihot_model_id_lookup'
        )

        multi_hot_feature = multi_hot_feature.apply(lambda x : self.__class__.multi_hot_same_shape(x, max_len=len(vocabulary)))
        multi_hot_feature = list(multi_hot_feature)

        multi_hot_layer = tf.keras.layers.StringLookup(vocabulary = vocabulary,
                                                    output_mode = "multi_hot",
                                                    num_oov_indices=1
                                                    )
        multi_hot_encodings = multi_hot_layer(multi_hot_feature)
        multi_hot_encodings = multi_hot_encodings[:, 1:]
        
        num_animes = multi_hot_encodings.shape[0]
        num_multi_hot_dims = multi_hot_encodings.shape[1]

        self.multi_hot_encoding_layer = tf.keras.layers.Embedding(
            num_animes,
            num_multi_hot_dims,
            embeddings_initializer=tf.keras.initializers.Constant(multi_hot_encodings),
            trainable = False,
            name = 'multi_hot_enconding_layer'
        )
    
    def call(self, anime_id):
        anime_idx = self.anime_id_lookup_layer(anime_id)
        anime_multihot_encoding = self.multi_hot_encoding_layer(anime_idx)
        return anime_multihot_encoding
    
    @staticmethod
    def multi_hot_same_shape(list_entities, max_len = 30):
        list_entities = list_entities[:max_len]
        num_add = max_len - list_entities.shape[0]
        return np.concatenate([list_entities , num_add * ["[UNK]"]])


In [16]:
class RetrievalModel(tfrs.Model):

    def __init__(self, anime_model, unique_anime_ids):
        super().__init__()
        
        self.anime_model = anime_model

        animes_ds = tf.data.Dataset.from_tensor_slices(unique_anime_ids)
        retrieval_metrics = tfrs.metrics.FactorizedTopK(
            candidates = animes_ds.batch(128).map(self.anime_model)
        )
        self.task = tfrs.tasks.Retrieval(
            metrics=retrieval_metrics
        )

    def compute_loss(self, features, training=False):
        
        anime_embeddings = self.anime_model(features["animeA"])
        positive_anime_embeddings = self.anime_model(features["animeB"])

        return self.task(anime_embeddings, positive_anime_embeddings, compute_metrics=not training)

In [17]:
anime_genre_model = AnimeMultiHotModel(anime_data['anime_id'], anime_data['genres'], anime_genre_vocab)
genre_retrieval_model = RetrievalModel(anime_genre_model, anime_data['anime_id'])
genre_retrieval_model.compile()
print(genre_retrieval_model.evaluate(val_ds, return_dict=True))

16/16 [==============================] - 6s 341ms/step - factorized_top_k/top_1_categorical_accuracy: 0.1333 - factorized_top_k/top_5_categorical_accuracy: 0.1480 - factorized_top_k/top_10_categorical_accuracy: 0.1663 - factorized_top_k/top_50_categorical_accuracy: 0.2668 - factorized_top_k/top_100_categorical_accuracy: 0.3478 - loss: 13950.4087 - regularization_loss: 0.0000e+00 - total_loss: 13950.4087
{'factorized_top_k/top_1_categorical_accuracy': 0.1332821100950241, 'factorized_top_k/top_5_categorical_accuracy': 0.1479717344045639, 'factorized_top_k/top_10_categorical_accuracy': 0.1663491129875183, 'factorized_top_k/top_50_categorical_accuracy': 0.2668100893497467, 'factorized_top_k/top_100_categorical_accuracy': 0.34778735041618347, 'loss': 12371.46875, 'regularization_loss': 0, 'total_loss': 12371.46875}
